# Travailler avec un jeu de données

## A) Préparation

### 1) On s'équipe des outils dont nous aurons besoin

- La bibliothèque d'analyse de données
- La bibliothèque destinée à tracer et visualiser des données sous forme de graphiques
- La bibliothèque de gestion des dates

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime
%matplotlib inline

### 2) Puis on se connecte à notre base de données

La base de données des participants:  
https://docs.google.com/a/tamedia.com/spreadsheets/d/1KtvS7DVQ5qzK6LHWt5npAcZBhBsTlTfJkzOdN7E03DA/edit?usp=sharing

#### Charge la base de données directement depuis Google

On remplace toutes les valeurs nulles - par exemple, si un utilisateur à répondu *aucun* dans la cellule *Nombre d'enfants* - par zéro.

In [ ]:
spreadsheet_url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vRwQgWrUdT_NYxOK7uSzYTbwU445CWe18-0M0KJIJFShWBf-jtzVU9B7PzKsV5yKbYULDNaNUlLtxmj/pub?gid=0&single=true&output=csv'

df = pd.read_csv(
    spreadsheet_url,
    error_bad_lines=False,
    na_values=[0, "Non", "non", "NoN", "NON", "Aucun", "aucun"])

#### Le jeu de données complet

In [ ]:
df

#### Montre les cinq premières lignes du jeu de données

In [ ]:
df.head()

In [ ]:
df["Prénom"]

In [ ]:
df[df["Signe astrologique"] == "Taureau"]

In [ ]:
df[df["Nombre d'enfants"] == 1]

In [ ]:
df[df["Prénom"].str.contains("na")]

### 3) On corrige les éventuelles erreurs et on convertit les données en un format que le système comprend

#### Les dates et chiffres sont transformés

In [ ]:
df["Date de naissance"] = pd.to_datetime(df["Date de naissance"])
df["Nombre d'enfants"].fillna(0, inplace=True)
df["Taille en cm"] = df["Taille en cm"].replace(r"[^0-9]", "", regex=True).astype(float)

#### Calcule l'âge de chacun d'entre nous

In [ ]:
now = pd.Timestamp(datetime.datetime.now())
df["Age"] = (now - df["Date de naissance"]).astype('<m8[Y]')

#### Vérifie le résultat des modifications

In [ ]:
df.head()

## B) Un aperçu rapide de notre jeu de données

### 1) Affiche le nombre de lignes et de colonnes de notre base données (Combien nous sommes et combien de paramètres nous caractérisent)

In [ ]:
df.shape

### 2) Affiche le nom de chaque paramètre qui nous caractérise ainsi que son type de donnée

In [ ]:
df.dtypes

### 3) Nous pourrions commencer par demander un aperçu statistique de notre groupe

In [ ]:
df.describe()

## C) Explorer le jeu de données pour y dénicher un sujet

### 1) Les hommes qui travaillent ont-ils plus d'enfants que les femmes dans la même situation?

#### Nombre d'hommes et de femmes

In [ ]:
df["Sexe"].value_counts()

#### Somme du nombre d'enfants des hommes et des femmes

In [ ]:
df.groupby("Sexe")["Nombre d'enfants"].sum()

#### Un résultat plus détaillé donnerait

In [ ]:
df.groupby("Sexe")["Nombre d'enfants"].value_counts()

#### ... et en pourcentage?

In [ ]:
df.groupby("Sexe")["Nombre d'enfants"].value_counts(normalize=True)

#### Que nous dit la moyenne d'enfants par groupes?

In [ ]:
df.groupby("Sexe")["Nombre d'enfants"].mean()

#### et le nombre médian d'enfants par groupes?

In [ ]:
df.groupby("Sexe")["Nombre d'enfants"].median()

### 2) Et si nous nous groupions par signe astrologique?

In [ ]:
df["Signe astrologique"].value_counts()

In [ ]:
df.groupby("Signe astrologique")["Nombre d'enfants"].sum()

#### Pour plus de lisibilité, nous pourrions afficher la liste précédente sous la forme d'un graphique

On groupe par signe astrologique, on demande la somme de chaque signe puis on classe le résultat par ordre décroissant avant de produire le graphique.

In [ ]:
df.groupby("Signe astrologique")["Nombre d'enfants"].sum().sort_values().plot(kind="barh")

#### Si nous voulions isoler un signe astrologique

In [ ]:
df[df['Signe astrologique'] == "Scorpion"]

### 3) Qui sont les plus jeunes? les plus vieux?

In [ ]:
df[["Prénom", "Age"]].sort_values(by="Age")

#### Quelles sont les tranches d'âge les plus représentées?

In [ ]:
df['Date de naissance'].dt.year.hist(bins=5)

#### Quels sont les mois les plus propices aux naissances?

In [ ]:
df['Date de naissance'].dt.month.hist(bins=12)

#### A la conception?

In [ ]:
for index, row in df.iterrows():
    print((row['Date de naissance'].month - 9) % 12)

In [ ]:
(pd.Series((df['Date de naissance'].dt.month - 9) % 12)
 .sort_values()
 .value_counts(sort=False)
 .plot(kind="barh"))

#### Que peut-on tirer des jours de naissance?

In [ ]:
df['Date de naissance'].dt.weekday.hist(bins=7)

#### Affichons un histogramme des âges divisés en cinq tranches

In [ ]:
df['Age'].hist(bins=5)

### Age moyen

In [ ]:
df['Age'].mean()

### Age médian

In [ ]:
df['Age'].median()

### Repartissons-nous en cinq groupes d'âge
Discrétiser notre série d'âges en 5 quantiles grâce à la fonction qcut

In [ ]:
pd.qcut(df['Age'], 5, labels=False, retbins=True)

#### La liste précédente ne nous donne pas d'autre information que l'index de la ligne et le numéro de son groupe.
Ajoutons ce numéro, qui représente un quintile, à notre jeu de données.

In [ ]:
df["Groupe d'âge"] = pd.qcut(df['Age'], 4, labels=False)
df.head()

#### Qui appartient au premier groupe d'âge (les plus jeunes)?

In [ ]:
df[df["Groupe d'âge"] == 0]

#### Les jeunes sont-ils plus grands que les moins jeunes?

In [ ]:
df.groupby("Groupe d'âge")["Taille en cm"].median().plot()

In [ ]:
df.rolling(12, on='Date de naissance', min_periods=1).mean().plot(x="Date de naissance", y="Taille en cm")

### 4) A partir de quel âge notre échantillon commence-t-il à fonder une famille?
Quel est le degré de corrélation entre les âges et le nombre d'enfants

In [ ]:
df.plot(kind="scatter", x="Age", y="Nombre d'enfants")

In [ ]:
colors_list = []
for index, row in df.iterrows():
    if row['Sexe'] == "f":
        colors_list.append("#ff3388")
    else:
        colors_list.append("#333333")

df.plot(kind="scatter", x="Age", y="Nombre d'enfants", color=colors_list)

### 5) Qui est susceptible de bien s'entendre avec qui?

#### Écart normalisé entre chaque valeur et la moyenne

In [ ]:
df_values = df[["Age", "Taille en cm", "Nombre d'enfants"]]
df_norm = (df_values - df_values.mean()) / (df_values.max() - df_values.min())
df_norm.head()

#### Ecart des âges classé par ordre croissant

In [ ]:
df_norm.sort_values(by="Age")

#### Difficile de lire le tableau ci-dessus. Essayons d'en faire un graphique.

In [ ]:
# Le canevas du graphique
fig, ax = plt.subplots()
heatmap = ax.pcolor(df_norm, cmap=plt.cm.Blues, alpha=0.8)

# Format
fig = plt.gcf()
fig.set_size_inches(2, 6)

# Masque le cadre
ax.set_frame_on(False)

# Positionne les ticks
ax.set_yticks(np.arange(df.shape[0]) + 0.5, minor=False)
ax.set_xticks(np.arange(df_norm.shape[1]) + 0.5, minor=False)

# Donne l'apparence d'un tableau
ax.invert_yaxis()
ax.xaxis.tick_top()

# Définit les labels
ax.set_xticklabels(["Age", "Taille en cm", "Nombre d'enfants"], minor=False)
ax.set_yticklabels(df["Prénom"], minor=False)

# Pivote les labels du dessus
plt.xticks(rotation=90)

ax.grid(False)

### 6) Qui sort du lot?

In [ ]:
moyenne = (abs(df_norm["Taille en cm"]) 
           + abs(df_norm["Nombre d'enfants"])
           + abs(df_norm["Age"])) / 3

moyenne.sort_values(ascending=False)

#### On veut des noms!

In [ ]:
df_moyenne = pd.DataFrame(moyenne)
df_moyenne['nom'] = df['Prénom']
df_moyenne.sort_values(by=0, ascending=False)